# How to build a LangGraph agent using Mosaic AI Agent Framework 

This notebook demonstrates how to author a LangGraph agent that's compatible with Mosaic AI Agent Framework features. In this notebook you learn to:
- Author a tool-calling LangGraph agent wrapped with `ChatAgent` with custom inputs and outputs
- Manually test the agent's output
- Log and deploy the agent

To learn more about authoring an agent using Mosaic AI Agent Framework, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/author-agent) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/create-chat-model)).

## Prerequisites

- Address all `TODO`s in this notebook.


### USE CASE: AI-driven Financial Analyst
Our AI-driven Financial Analyst is designed to assist with various financial transactions and analyses. This intelligent agent can perform two primary functions:

Financial Calculations: The agent can execute Python code to perform a wide range of financial calculations, from simple interest computations to complex investment analyses.

Email Analysis for Fraud Detection: The agent can scan through customer emails to identify important communications related to financial transactions, particularly focusing on potential fraudulent activities or confirming legitimate transactions that were initially flagged as suspicious.

In [0]:
%pip install -U -qqqq mlflow langchain langgraph==0.2.74 unitycatalog-langchain[databricks] databricks-langchain pydantic databricks-agents uv
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires numpy<1.24,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.6 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:

# catalog = "sarbanimaiti_catalog"
# schema = dbName = db = "dbdemos_fsi_fraud_detection"

# spark.sql(f"USE CATALOG {catalog}")
# spark.sql(f"USE SCHEMA {schema}")


## Define the agent in code 
([MLFLOW - Model From Code GUIDE](https://mlflow.org/docs/latest/model/models-from-code.html))
Define the agent code in a single cell below. This lets you easily write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.

#### Agent tools
This agent code adds the built-in Unity Catalog function `system.ai.python_exec` to the agent. The agent code also includes commented-out sample code for adding a vector search index to perform unstructured data retrieval.

For more examples of tools to add to your agent, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool))

#### Wrap the LangGraph agent using the `ChatAgent` interface

For compatibility with Databricks AI features, the `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent. This example uses the provided convenience APIs [`ChatAgentState`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentState) and [`ChatAgentToolNode`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentToolNode) for ease of use.

Databricks recommends using `ChatAgent` as it simplifies authoring multi-turn conversational agents using an open source standard. See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).


**Authoring your agent using ChatAgent provides the following benefits:**

- **Advanced agent capabilities**
> - **Streaming output**: Enable interactive user experiences by streaming output in smaller chunks.
> - **Comprehensive tool-calling message history**: Return multiple messages, including intermediate tool-calling messages, for improved quality and conversation management.
> - **Tool-calling confirmation suppor**t
> - **Multi-agent system support**

**Streamlined development, deployment, and monitoring**
> - **Databricks feature integration**: Out-of-the-box compatibility with AI Playground, Agent Evaluation, and Agent Monitoring.
> - **Typed authoring interfaces**: Write agent code using typed Python classes, benefiting from IDE and notebook autocomplete.
> - **Automatic signature inference**: MLflow automatically infers ChatAgent signatures when logging the agent, simplifying registration and deployment. See Infer Model Signature during logging.
> - **AI Gateway-enhanced inference tables**: AI Gateway inference tables are automatically enabled for deployed agents, providing access to detailed request log metadata.


#### Custom inputs and outputs

Some scenarios may require additional agent inputs, such as client_type and session_id, or outputs like retrieval source links that should not be included in the chat history for future interactions.

For these scenarios, MLflow ChatAgent natively supports the fields custom_inputs and custom_outputs.
The agent is designed to handle custom inputs and outputs using the `predict` methods and the `add_custom_outputs` function.

- In the `predict` and `predict_stream` methods of the `LangGraphChatAgent` class:
    - Custom inputs are passed as an optional parameter and included in the request dictionary.
    - Custom outputs are captured from the agent's response and added to the `ChatAgentResponse` object.

- In the `add_custom_outputs` function:
    - This function is added as a node in the agent's workflow to append custom outputs to the state before returning the final response.

In [0]:
%%writefile agent.py
from typing import Any, Generator, Optional, Sequence, Union

import mlflow
from databricks_langchain import ChatDatabricks, VectorSearchRetrieverTool
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

mlflow.langchain.autolog()

client = DatabricksFunctionClient()
set_uc_function_client(client)

############################################
# Define your LLM endpoint and system prompt
############################################
# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME =  "agents-demo-gpt4o"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)


# TODO: Update with your system prompt
system_prompt = """
You are a helpful assistant. You can assist with Financial calculations and customer email analysis for financial transactions.
Use tool "system.ai.python_exec" for running python code to perform any financial calculation.
Use tool "sarbanimaiti_catalog.dbdemos_fsi_fraud_detection.cust_emails" to extract customer emails reporting any fraudulent transactions or emails confirming legitimate transactions that were initially flagged as suspicious. These emails will be used  for fraud analysis & investigation
"""

###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/en/generative-ai/agent-framework/agent-tool.html
###############################################################################
tools = []

# You can use UDFs in Unity Catalog as agent tools
# Below, we add the `system.ai.python_exec` UDF, which provides
# a python code interpreter tool to our agent

# TODO: Add additional tools
uc_tool_names = ["system.ai.python_exec"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)

# Use Databricks vector search indexes as tools
# See https://docs.databricks.com/en/generative-ai/agent-framework/unstructured-retrieval-tools.html#locally-develop-vector-search-retriever-tools-with-ai-bridge
# for details

# TODO: Add vector search indexes : I am adding my own Vector Search Retriever Tool 
# (Search with customer name :'Steven Marshall'  or Transaction id 'cd506e3e-4809-45dc-b5b9-debfe70cf2f4')

# vector_search_tools = [
#         VectorSearchRetrieverTool(
#         index_name="",
#         # filters="..."
#     )
# ]
vector_search_tools = [
        VectorSearchRetrieverTool(
        index_name="sarbanimaiti_catalog.dbdemos_fsi_fraud_detection.cust_emails_idx",
        # filters="..."
        query_type="HYBRID", # Query type ("ANN" or "HYBRID").
        tool_name="cust_emails", # Used by the LLM to understand the purpose of the tool
        tool_description="Use this tool to extract customer emails reporting fraudulent transactions or emails confirming legitimate transactions that were initially flagged as suspicious. These emails will be used  for fraud analysis & investigation", # Used by the LLM to understand the purpose of the tool
    )
]
tools.extend(vector_search_tools)

#####################
## Define agent logic
#####################


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolExecutor, Sequence[BaseTool]],
    agent_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)

    # Define the function that determines which node to go to
    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there are function calls, continue. else, end
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    if agent_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": agent_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | model

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        response = model_runnable.invoke(state, config)

        return {"messages": [response]}

    def add_custom_outputs(state: ChatAgentState):
        # TODO: Return extra content with the custom_outputs key before returning
        return {
            "messages": [{"role": "assistant", "content": "Adding custom outputs"}],
            "custom_outputs": {
                **(state.get("custom_outputs") or {}),
                **(state.get("custom_inputs") or {}),
                "key": "value",
            }, 
        }


    workflow = StateGraph(ChatAgentState)

    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))
    workflow.add_node("add_custom_outputs", RunnableLambda(add_custom_outputs))

    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "end": "add_custom_outputs",
        },
    )
    workflow.add_edge("tools", "agent")
    workflow.add_edge("add_custom_outputs", END)

    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        # TODO: Use context and custom_inputs to alter the behavior of the agent
        request = {
            "messages": self._convert_messages_to_dict(messages),
            **({"custom_inputs": custom_inputs} if custom_inputs else {}),
            **({"context": context.model_dump_compat()} if context else {}),
        }

        response = ChatAgentResponse(messages=[])
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                if not node_data:
                    continue
                for msg in node_data.get("messages", []):
                    response.messages.append(ChatAgentMessage(**msg))
                if "custom_outputs" in node_data:
                    response.custom_outputs = node_data["custom_outputs"]
        return response

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        # TODO: Use context and custom_inputs to alter the behavior of the agent
        request = {
            "messages": self._convert_messages_to_dict(messages),
            **({"custom_inputs": custom_inputs} if custom_inputs else {}),
            **({"context": context.model_dump_compat()} if context else {}),
        }

        last_message = None
        last_custom_outputs = None

        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                if not node_data:
                    continue
                messages = node_data.get("messages", [])
                custom_outputs = node_data.get("custom_outputs")

                for message in messages:
                    if last_message:
                        yield ChatAgentChunk(delta=last_message)
                    last_message = message
                if custom_outputs:
                    last_custom_outputs = custom_outputs
        if last_message:
            yield ChatAgentChunk(delta=last_message, custom_outputs=last_custom_outputs)


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)

Overwriting agent.py


## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from agent import AGENT

AGENT.predict(
    {
        "messages": [{"role": "user", "content": "Hello!"}],
        "custom_inputs": {"key": "value"},
    }
)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-3d326389-a308-476c-9cc5-f627c89254ce/lib/python3.11/site-packages/databricks_langchain/uc_ai/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


ChatAgentResponse(messages=[ChatAgentMessage(role='assistant', content='Hi there! How can I assist you today?', name=None, id='run-dbdfd72a-a3eb-40a3-9042-e445544eead5-0', tool_calls=None, tool_call_id=None, attachments=None), ChatAgentMessage(role='assistant', content='Adding custom outputs', name=None, id='a434cdba-7619-42f2-98aa-13509428b187', tool_calls=None, tool_call_id=None, attachments=None)], finish_reason=None, custom_outputs={'key': 'value'}, usage=None)

Trace(request_id=tr-ea038594c75945e5864f28456504140b)

In [0]:
for event in AGENT.predict_stream(
    {
        "messages": [{"role": "user", "content": "Calculate the final amount on investment (10000 * (1 + 0.05) * 2) in python"}],
        "custom_inputs": {"key": "value"},
    }
):
    print(event, "-----------\n")

delta=ChatAgentMessage(role='assistant', content='', name=None, id='run-e0486078-52fa-422f-b737-452033da0b31-0', tool_calls=[ToolCall(id='call_CGdMs19amQwWI5pVPDvYyrR9', type='function', function=Function(name='system__ai__python_exec', arguments='{"code": "# Calculate the final amount on investment\\ninitial_investment = 10000\\ninterest_rate = 0.05\\nmultiplier = 2\\n\\nfinal_amount = initial_investment * (1 + interest_rate) * multiplier\\nfinal_amount"}'))], tool_call_id=None, attachments=None) finish_reason=None custom_outputs=None usage=None -----------

delta=ChatAgentMessage(role='tool', content='{"format": "SCALAR", "value": ""}', name='system__ai__python_exec', id='983b0787-b7b1-482d-bbd6-62e7efb552c0', tool_calls=None, tool_call_id='call_CGdMs19amQwWI5pVPDvYyrR9', attachments=None) finish_reason=None custom_outputs=None usage=None -----------

delta=ChatAgentMessage(role='assistant', content='It seems there was an error in the calculation. Let me correct it and try again.', n

Trace(request_id=tr-ea911188c051444da93a097b202050a7)

In [0]:
for event in AGENT.predict_stream(
    {
        "messages": [{"role": "user", "content": "Is there any email communication from Steven Marshall?"}],
        "custom_inputs": {"key": "value"},
    }
):
    print(event, "-----------\n")

delta=ChatAgentMessage(role='assistant', content='', name=None, id='run-b5c1393d-ade1-493b-be15-819e3283e2c1-0', tool_calls=[ToolCall(id='call_ZeEGkUZJt8lbOSUU1Vvabzxv', type='function', function=Function(name='system__ai__python_exec', arguments='{"code": "# Since I don\'t have access to the actual email database or any external systems, I will simulate the process of checking for emails from a specific person.\\n# Let\'s assume we have a function that can check for emails from a specific person in a database.\\n\\ndef check_for_emails_from_person(person_name):\\n    # Simulated database of emails\\n    email_database = [\\n        {\'from\': \'steven.marshall@example.com\', \'subject\': \'Fraudulent Transaction Alert\', \'body\': \'Please investigate this transaction.\'},\\n        {\'from\': \'jane.doe@example.com\', \'subject\': \'Account Update\', \'body\': \'I have updated my account details.\'},\\n        {\'from\': \'steven.marshall@example.com\', \'subject\': \'Re: Fraudulent 

Trace(request_id=tr-2e5feb5e16df446db0c97f877dec1da5)

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`

  - **TODO**: If your Unity Catalog tool queries a [vector search index](docs link) or leverages [external functions](docs link), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See docs ([AWS](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).



In [0]:
import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))


with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        pip_requirements=[
            "mlflow",
            "langchain",
            "langgraph==0.2.74",
            "databricks-langchain",
            "unitycatalog-langchain[databricks]",
            "pydantic",
        ],
        resources=resources,
    )

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2896476171232787>, line 1
----> 1 from agent import AGENT
      3 AGENT.predict(
      4     {
      5         "messages": [{"role": "user", "content": "Hello!"}],
      6         "custom_inputs": {"key": "value"},
      7     }
      8 )

File /Workspace/Users/sarbani.maiti@databricks.com/my-llm-agent-project/Jan-2025-agent-proj/AI-Agent-Authoring-patterns/agent.py:42
     39 catalog = "sarbanimaiti_catalog"
     40 schema = dbName = db = "dbdemos_fsi_fraud_detection"
---> 42 spark.sql(f"USE CATALOG {catalog}")
     43 spark.sql(f"USE SCHEMA {schema}")
     45 # TODO: Update with your system prompt

NameError: name 'spark' is not defined

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2896476171232787>, line 1
----> 1 from agent import AGENT
      3 AGENT.predict(
      4     {
      5         "messages": [{"role": "user", "content": "Hello!"}],
      6         "custom_inputs": {"key": "value"},
      7     }
      8 )

File /Workspace/Users/sarbani.maiti@databricks.com/my-llm-agent-project/Jan-2025-agent-proj/AI-Agent-Authoring-patterns/agent.py:42
     39 catalog = "sarbanimaiti_catalog"
     40 schema = dbName = db = "dbdemos_fsi_fraud_detection"
---> 42 spark.sql(f"USE CATALOG {catalog}")
     43 spark.sql(f"USE SCHEMA {schema}")
     45 # TODO: Update with your system prompt

NameError: name 'spark' is not defined

## Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.

- **TODO** Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "sarbanimaiti_catalog"
schema = "dbdemo_agent"
model_name = "lgtc_custsch_Finagnt"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2896476171232787>, line 1
----> 1 from agent import AGENT
      3 AGENT.predict(
      4     {
      5         "messages": [{"role": "user", "content": "Hello!"}],
      6         "custom_inputs": {"key": "value"},
      7     }
      8 )

File /Workspace/Users/sarbani.maiti@databricks.com/my-llm-agent-project/Jan-2025-agent-proj/AI-Agent-Authoring-patterns/agent.py:42
     39 catalog = "sarbanimaiti_catalog"
     40 schema = dbName = db = "dbdemos_fsi_fraud_detection"
---> 42 spark.sql(f"USE CATALOG {catalog}")
     43 spark.sql(f"USE SCHEMA {schema}")
     45 # TODO: Update with your system prompt

NameError: name 'spark' is not defined

## Deploy the agent

In [0]:


# from databricks import agents
# agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "docs"})

# catalog = "sarbanimaiti_catalog"
# schema = "dbdemo_agent"
model_name = "lgtc_custsch_Finagnt"
UC_MODEL_NAME = "sarbanimaiti_catalog.dbdemo_agent.lgtc_custsch_Finagnt"


from databricks import agents
agents.deploy(UC_MODEL_NAME, 3, tags = {"endpointSource": "docs"})

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2896476171232787>, line 1
----> 1 from agent import AGENT
      3 AGENT.predict(
      4     {
      5         "messages": [{"role": "user", "content": "Hello!"}],
      6         "custom_inputs": {"key": "value"},
      7     }
      8 )

File /Workspace/Users/sarbani.maiti@databricks.com/my-llm-agent-project/Jan-2025-agent-proj/AI-Agent-Authoring-patterns/agent.py:42
     39 catalog = "sarbanimaiti_catalog"
     40 schema = dbName = db = "dbdemos_fsi_fraud_detection"
---> 42 spark.sql(f"USE CATALOG {catalog}")
     43 spark.sql(f"USE SCHEMA {schema}")
     45 # TODO: Update with your system prompt

NameError: name 'spark' is not defined

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)).

#### Test Index

In [0]:
import databricks.sdk.service.catalog as c
from databricks.vector_search.client import VectorSearchClient
VECTOR_SEARCH_ENDPOINT_NAME = "one-env-shared-endpoint-1"
vsc = VectorSearchClient(disable_notice=True)

# Where we want to store our index
vs_index_fullname = f"{catalog}.{db}.cust_emails_idx"
#embedding_model_endpoint = "databricks-gte-large-en"

# index = vsc.create_delta_sync_index(
#   endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
#   source_table_name=f"{catalog}.{db}.customer_emails", 
#   index_name=vs_index_fullname,
#   pipeline_type="TRIGGERED",
#   primary_key='transaction_id',
#   embedding_source_column="email_content",
#   embedding_model_endpoint_name= embedding_model_endpoint)
index = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)

# if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
#   print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
#   index = vsc.create_delta_sync_index(
#     endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
#     source_table_name=f"{catalog}.{db}.customer_emails", 
#     index_name=vs_index_fullname,
#     pipeline_type="TRIGGERED",
#     primary_key='transaction_id',
#     embedding_source_column="email_content",
#     embedding_model_endpoint_name= embedding_model_endpoint
#   )
# else:
#   print(f"Grabbing existing index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
#   index = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2896476171232787>, line 1
----> 1 from agent import AGENT
      3 AGENT.predict(
      4     {
      5         "messages": [{"role": "user", "content": "Hello!"}],
      6         "custom_inputs": {"key": "value"},
      7     }
      8 )

File /Workspace/Users/sarbani.maiti@databricks.com/my-llm-agent-project/Jan-2025-agent-proj/AI-Agent-Authoring-patterns/agent.py:42
     39 catalog = "sarbanimaiti_catalog"
     40 schema = dbName = db = "dbdemos_fsi_fraud_detection"
---> 42 spark.sql(f"USE CATALOG {catalog}")
     43 spark.sql(f"USE SCHEMA {schema}")
     45 # TODO: Update with your system prompt

NameError: name 'spark' is not defined

In [0]:

index.similarity_search(
    columns=['customer_id', 'transaction_id', 'email_content'],
    query_text='83214a7c-d15f-4d5c-8708-c25058fd3423',
    query_type="hybrid"
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2896476171232787>, line 1
----> 1 from agent import AGENT
      3 AGENT.predict(
      4     {
      5         "messages": [{"role": "user", "content": "Hello!"}],
      6         "custom_inputs": {"key": "value"},
      7     }
      8 )

File /Workspace/Users/sarbani.maiti@databricks.com/my-llm-agent-project/Jan-2025-agent-proj/AI-Agent-Authoring-patterns/agent.py:42
     39 catalog = "sarbanimaiti_catalog"
     40 schema = dbName = db = "dbdemos_fsi_fraud_detection"
---> 42 spark.sql(f"USE CATALOG {catalog}")
     43 spark.sql(f"USE SCHEMA {schema}")
     45 # TODO: Update with your system prompt

NameError: name 'spark' is not defined